In [ ]:
from orca_dataloader import *



wsi_image_file = '../../datasets/ORCA/trainning/tumor/wsi/TCGA-4P-AA8J-01Z-00-DX1.5B44796F-D099-4076-9CAF-B40C2B83F432_0.png'
#wsi_image_file = '../../datasets/ORCA/trainning/tumor/wsi/TCGA-BA-5151-01Z-00-DX1.32EE476B-128D-4153-A57C-CFA823B993A2_0.png'

wsi_image = open_wsi(wsi_image_file)
print(wsi_image)

wsi_image_pil = scale_down_wsi(wsi_image, 0.625, use_openslide_propeties=False)
wsi_image_np = pil_to_np(wsi_image_pil)
show_np_img(wsi_image_np)
print(wsi_image_np.shape)

#img = Image.open(wsi_image_file)
#img.save(wsi_image_file.replace(".png", ".tiff"))

In [ ]:
from orca_dataloader import *

magnification=0.625
wsi_image_file = '../../datasets/ORCA/trainning/tumor/wsi/TCGA-4P-AA8J-01Z-00-DX1.5B44796F-D099-4076-9CAF-B40C2B83F432_0.png'
wsi_mask_file = '../../datasets/ORCA/trainning/lesion_annotations/TCGA-4P-AA8J-01Z-00-DX1.5B44796F-D099-4076-9CAF-B40C2B83F432_0_mask.png'


wsi_image = open_wsi(wsi_image_file)
wsi_image_pil = load_pil_image(wsi_image_file, gray=False)
wsi_mask_pil = load_pil_image(wsi_mask_file, gray=True)

pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, use_openslide_propeties=False)
np_scaled_down_image = pil_to_np(pil_scaled_down_image)

np_tumor_mask = np.zeros((wsi_mask_pil.size[0], wsi_mask_pil.size[1]), dtype=bool)
np_tumor_mask[pil_to_np(wsi_mask_pil) == 255] = True

wsi_image_np = pil_to_np(wsi_image_pil)
np_tumor_masked = mask_rgb(wsi_image_np, np_tumor_mask)
np_tumor_masked = image_resize(np_tumor_masked, height=np_scaled_down_image.shape[1])

show_np_img(np_scaled_down_image)
show_np_img(np_tumor_masked)

# Extracts the normal image-patches dataset

In [7]:
from orca_dataloader import *

magnification=0.625
scale = get_scale_by_magnification(magnification)
tile_size=20
tile_size_original = int(scale * tile_size)


cls = "tumor"
dataset_types = ["training"]#, "testing"]
for dataset_type in sorted(dataset_types):
    
    logger.info("{} images".format(dataset_type))
    
    orca_dir = "../../datasets/ORCA"
    annotations_dir = "{}/{}/lesion_annotations".format(orca_dir, dataset_type)
    wsi_images_dir = "{}/{}/{}/wsi".format(orca_dir, dataset_type, cls)
    patch_images_dir = "{}/{}/{}/patch/640x640".format(orca_dir, dataset_type, cls)
    
    for r, d, f in sorted(os.walk(wsi_images_dir)):
        for wsi_file in sorted(f):
            
            wsi_image_file = "{}/{}".format(r, wsi_file)
            wsi_image_number = wsi_file.replace(".png", "")

            file_is_png = wsi_image_file.lower().endswith('.png')
            if file_is_png:

                logger.info("Processing wsi '{}'".format(wsi_file))
                
                # check directory to save image-patches
                dir_to_save = "{}/{}".format(patch_images_dir, wsi_image_number)
                if not os.path.exists(dir_to_save):
                    os.makedirs("{}/01-roi/01-original".format(dir_to_save))
                    os.makedirs("{}/01-roi/02-mask".format(dir_to_save))
                    os.makedirs("{}/01-roi/03-roi".format(dir_to_save))             
                    os.makedirs("{}/02-non_roi/01-original".format(dir_to_save))
                    os.makedirs("{}/02-non_roi/02-mask".format(dir_to_save))
                
                # tumor annotations mask
                annotation_file = "{}/{}_mask.png".format(annotations_dir, wsi_image_number)
                wsi_mask_pil = load_pil_image(annotation_file, gray=True)
                
                # tumor tissue region
                wsi_image = open_wsi(wsi_image_file)
                wsi_image_pil = load_pil_image(wsi_image_file, gray=False)
                pil_scaled_down_image = scale_down_wsi(wsi_image, magnification, False)
                
                np_tumor_mask = np.zeros((wsi_mask_pil.size[0], wsi_mask_pil.size[1]), dtype=bool)
                np_tumor_mask[pil_to_np(wsi_mask_pil) == 255] = True
                pil_tumor_mask = np_to_pil(np_tumor_mask)
                
                np_normal_mask = np.zeros((wsi_mask_pil.size[0], wsi_mask_pil.size[1]), dtype=bool)
                np_normal_mask[pil_to_np(wsi_mask_pil) < 255] = True
                np_normal_mask[pil_to_np(wsi_mask_pil) == 0] = False
                pil_normal_mask = np_to_pil(np_normal_mask)
                
                pil_background = np_to_pil(np_normal_mask | np_tumor_mask)

                wsi_image_np = pil_to_np(wsi_image_pil)
                np_tumor_masked = mask_rgb(wsi_image_np, np_tumor_mask)
                np_tumor_masked = image_resize(np_tumor_masked, height=pil_scaled_down_image.size[1])
                pil_tumor_masked = np_to_pil(np_tumor_masked)
                
                np_normal_masked = mask_rgb(wsi_image_np, np_normal_mask)
                np_normal_masked = image_resize(np_normal_masked, height=pil_scaled_down_image.size[1])
                pil_normal_masked = np_to_pil(np_normal_masked)

                blend_tumor = blend_image(wsi_image_pil, pil_tumor_mask, foreground='red', inverse=True)
                blend_tissue = blend_image(blend_tumor, pil_normal_mask, foreground='green', inverse=True)
                blend_background = blend_image(blend_tissue, pil_background, foreground='blue', inverse=False).resize(pil_tumor_masked.size, Image.ANTIALIAS)

                pil_scaled_down_image.save('{}/{}_1.png'.format(dir_to_save, wsi_image_number))
                blend_background.save('{}/{}_2.png'.format(dir_to_save, wsi_image_number))
                pil_normal_masked.save('{}/{}_3.png'.format(dir_to_save, wsi_image_number))
                pil_tumor_masked.save('{}/{}_4.png'.format(dir_to_save, wsi_image_number))
                
                #heat grid normal
                pil_img_normal_result, heat_grid_normal, number_of_tiles_normal = draw_heat_grid(np_normal_masked, tile_size)
                pil_img_normal_result.save('{}/{}_5.png'.format(dir_to_save, wsi_image_number))

                # heat grid tumor
                pil_img_result, heat_grid, number_of_tiles = draw_heat_grid(np_tumor_masked, tile_size)
                pil_img_result.save('{}/{}_6.png'.format(dir_to_save, wsi_image_number))
                
                used_patches = set()                
                tiles_tumor_tissue = []
                tiles_normal_tissue = []
                tiles_background = []
                for idx, (position, row, column, location, size, color) in enumerate(heat_grid):

                    tile = (position, row, column, location, size, color)                    
                    if color == YELLOW_COLOR or color == ORANGE_COLOR or color == RED_COLOR:
                        tiles_tumor_tissue.append(tile)
                        used_patches.add("r{}c{}".format(row, column))
                
                for idx, (position, row, column, location, size, color) in enumerate(heat_grid_normal):

                    name = "r{}c{}".format(row, column)
                    tile = (position, row, column, location, size, color)
                    if name not in used_patches:
                        if color == GREEN_COLOR:
                            tiles_background.append(tile)
                        else:
                            tiles_normal_tissue.append(tile)
                
                logger.info("\t {} patches of 640x640 size.".format(len(heat_grid_normal)))
                logger.info("\t\t {} patches of 640x640 (normal tissue).".format(len(tiles_normal_tissue)))
                logger.info("\t\t {} patches of 640x640 (tumor tissue).".format(len(tiles_tumor_tissue)))
                logger.info("\t\t {} patches of 640x640 (background).".format(len(tiles_background)))
                
                
                # extract and save normal patches
                for (position, row, column, location, size, color) in tiles_normal_tissue:
                    
                    r_s = row * tile_size_original
                    r_e = r_s + tile_size_original
                    c_s = column * tile_size_original
                    c_e = c_s + tile_size_original
                    np_tile_masked = np_tumor_masked[r_s:r_e, c_s:c_e]
                    np_tile_mask = np_tumor_mask[r_s:r_e, c_s:c_e]
                                        
                    # only tile with valid size
                    if np_tile_mask.shape[0] == tile_size_original and np_tile_mask.shape[1] == tile_size_original:

                        tile_pil, tile_np = read_region(wsi_image_file, column, row, magnification, tile_size)
                        left = (column * tile_size_original)
                        top = (row * tile_size_original)

                        pil_mask = np_to_pil(np.zeros((tile_np.shape[0], tile_np.shape[1]), dtype=np.uint8))
                        np_tile_mask = pil_to_np(pil_mask).astype(bool)

                        # save the extracted normal tile
                        tile_pil.save('{}/02-non_roi/{}/{}_r{}c{}.png'.format(dir_to_save, "01-original", wsi_image_number, row, column))
                        np_to_pil(np_tile_mask).save('{}/02-non_roi/{}/{}_r{}c{}.png'.format(dir_to_save, "02-mask", wsi_image_number, row, column))

                # extract and save tumor patches
                for (position, row, column, location, size, color) in tiles_tumor_tissue:
                    
                    r_s = row * tile_size_original
                    r_e = r_s + tile_size_original
                    c_s = column * tile_size_original
                    c_e = c_s + tile_size_original
                    np_tile_masked = np_tumor_masked[r_s:r_e, c_s:c_e]
                    np_tile_mask = np_tumor_mask[r_s:r_e, c_s:c_e]
                                        
                    # only tile with valid size
                    if np_tile_mask.shape[0] == tile_size_original and np_tile_mask.shape[1] == tile_size_original:

                        tile_pil, tile_np = read_region(wsi_image_file, column, row, magnification, tile_size)
                        left = (column * tile_size_original)
                        top = (row * tile_size_original)
                                                
                        pil_tile_roi = blend_image(np_to_pil(tile_np), np_to_pil(np_tile_mask), foreground='blue', inverse=True)
                        #pil_tile_roi = blend_image(pil_tile_roi, np_to_pil(np_tile_mask), foreground='green', inverse=False)
                        
                         # save the extracted tumor image-patch
                        tile_pil.save('{}/01-roi/{}/{}_r{}c{}.png'.format(dir_to_save, "01-original", wsi_image_number, row, column))
                        np_to_pil(np_tile_mask).save('{}/01-roi/{}/{}_r{}c{}.png'.format(dir_to_save, "02-mask", wsi_image_number, row, column))
                        pil_tile_roi.save('{}/01-roi/{}/{}_r{}c{}.png'.format(dir_to_save, "03-roi", wsi_image_number, row, column))
                

2021-01-24 21:03:26,954 :: INFO <module> :: training images
2021-01-24 21:03:26,958 :: INFO <module> :: Processing wsi 'TCGA-4P-AA8J-01Z-00-DX1.5B44796F-D099-4076-9CAF-B40C2B83F432_0.png'
2021-01-24 21:03:30,615 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:03:30,616 :: INFO <module> :: 		 3 patches of 640x640 (normal tissue).
2021-01-24 21:03:30,617 :: INFO <module> :: 		 42 patches of 640x640 (tumor tissue).
2021-01-24 21:03:30,623 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:04:08,290 :: INFO <module> :: Processing wsi 'TCGA-4P-AA8J-01Z-00-DX1.5B44796F-D099-4076-9CAF-B40C2B83F432_1.png'
2021-01-24 21:04:12,327 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:04:12,328 :: INFO <module> :: 		 4 patches of 640x640 (normal tissue).
2021-01-24 21:04:12,329 :: INFO <module> :: 		 41 patches of 640x640 (tumor tissue).
2021-01-24 21:04:12,330 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:04:48,253 :: INFO 

2021-01-24 21:13:03,760 :: INFO <module> :: Processing wsi 'TCGA-BA-5555-01Z-00-DX1.cffe9b77-36d4-49f1-86f8-90686c695985_0.png'
2021-01-24 21:13:07,705 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:13:07,707 :: INFO <module> :: 		 7 patches of 640x640 (normal tissue).
2021-01-24 21:13:07,708 :: INFO <module> :: 		 38 patches of 640x640 (tumor tissue).
2021-01-24 21:13:07,709 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:13:42,849 :: INFO <module> :: Processing wsi 'TCGA-BA-5555-01Z-00-DX1.cffe9b77-36d4-49f1-86f8-90686c695985_1.png'
2021-01-24 21:13:46,568 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:13:46,569 :: INFO <module> :: 		 14 patches of 640x640 (normal tissue).
2021-01-24 21:13:46,570 :: INFO <module> :: 		 31 patches of 640x640 (tumor tissue).
2021-01-24 21:13:46,571 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:14:21,903 :: INFO <module> :: Processing wsi 'TCGA-BA-5556-01Z-00-DX1.6941a8f

2021-01-24 21:25:05,138 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:25:05,139 :: INFO <module> :: 		 2 patches of 640x640 (normal tissue).
2021-01-24 21:25:05,140 :: INFO <module> :: 		 43 patches of 640x640 (tumor tissue).
2021-01-24 21:25:05,141 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:25:41,141 :: INFO <module> :: Processing wsi 'TCGA-BA-A6D8-01Z-00-DX1.0198253B-84AC-484E-9A58-F7CF8AEBE922_0.png'
2021-01-24 21:25:44,728 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:25:44,729 :: INFO <module> :: 		 9 patches of 640x640 (normal tissue).
2021-01-24 21:25:44,730 :: INFO <module> :: 		 36 patches of 640x640 (tumor tissue).
2021-01-24 21:25:44,731 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:26:20,488 :: INFO <module> :: Processing wsi 'TCGA-BA-A6D8-01Z-00-DX1.0198253B-84AC-484E-9A58-F7CF8AEBE922_1.png'
2021-01-24 21:26:24,003 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:26:2

2021-01-24 21:37:02,268 :: INFO <module> :: 		 4 patches of 640x640 (normal tissue).
2021-01-24 21:37:02,269 :: INFO <module> :: 		 41 patches of 640x640 (tumor tissue).
2021-01-24 21:37:02,270 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:37:38,478 :: INFO <module> :: Processing wsi 'TCGA-BB-4217-01Z-00-DX1.E351C8DB-F6CB-4327-9FEC-B9001E70C913_1.png'
2021-01-24 21:37:42,058 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:37:42,059 :: INFO <module> :: 		 10 patches of 640x640 (normal tissue).
2021-01-24 21:37:42,059 :: INFO <module> :: 		 35 patches of 640x640 (tumor tissue).
2021-01-24 21:37:42,060 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:38:16,654 :: INFO <module> :: Processing wsi 'TCGA-BB-4223-01Z-00-DX1.8806b83e-910f-4ee8-aa83-f84e3d13155f_0.png'
2021-01-24 21:38:19,979 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:38:19,980 :: INFO <module> :: 		 14 patches of 640x640 (normal tissue).
2021-

2021-01-24 21:47:33,834 :: INFO <module> :: 		 20 patches of 640x640 (tumor tissue).
2021-01-24 21:47:33,835 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:48:07,688 :: INFO <module> :: Processing wsi 'TCGA-BB-7866-01Z-00-DX1.144872E0-8C08-42CC-ACDD-45D28F35517D_1.png'
2021-01-24 21:48:11,207 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:48:11,208 :: INFO <module> :: 		 3 patches of 640x640 (normal tissue).
2021-01-24 21:48:11,209 :: INFO <module> :: 		 42 patches of 640x640 (tumor tissue).
2021-01-24 21:48:11,210 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:48:47,109 :: INFO <module> :: Processing wsi 'TCGA-BB-7870-01Z-00-DX1.6b49c1c8-19c4-4e0c-b4b5-f7e9e1ba782c_0.png'
2021-01-24 21:48:50,678 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:48:50,679 :: INFO <module> :: 		 10 patches of 640x640 (normal tissue).
2021-01-24 21:48:50,679 :: INFO <module> :: 		 35 patches of 640x640 (tumor tissue).
2021-0

2021-01-24 21:57:14,404 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:57:50,537 :: INFO <module> :: Processing wsi 'TCGA-BB-A6UM-01Z-00-DX1.403D4D0A-83F4-4D2F-9CBB-C7A1470AC565_1.png'
2021-01-24 21:57:54,085 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:57:54,086 :: INFO <module> :: 		 10 patches of 640x640 (normal tissue).
2021-01-24 21:57:54,088 :: INFO <module> :: 		 35 patches of 640x640 (tumor tissue).
2021-01-24 21:57:54,089 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-24 21:58:29,522 :: INFO <module> :: Processing wsi 'TCGA-BB-A6UO-01Z-00-DX1.11D049DC-EFC3-47EB-B390-A694BFD304A2_0.png'
2021-01-24 21:58:32,964 :: INFO <module> :: 	 49 patches of 640x640 size.
2021-01-24 21:58:32,965 :: INFO <module> :: 		 12 patches of 640x640 (normal tissue).
2021-01-24 21:58:32,966 :: INFO <module> :: 		 33 patches of 640x640 (tumor tissue).
2021-01-24 21:58:32,967 :: INFO <module> :: 		 4 patches of 640x640 (background).
2021-01-